In [1]:
import osmnx as ox
import momepy
import geopandas as gpd
import shapely
from shapely.geometry import LineString
from shapely.ops import unary_union
import os

/home/virgilxw/miniconda3/lib/python3.10/site-packages/osmnx/projection.py:3: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


In [2]:
# Set the local coordinate reference system to EPSG 3414 (which is the projected CRS used in singapore)
local_crs = 3414

# Define the place of interest as a string variable
place = "singapore"

# Define the latitude and longitude coordinates of the center point of the study area as a tuple
latlng = (1.29, 103.85)

# Define the distance in meters from the center point that the study area will cover
dist = 30000

# Read in the study area polygon shapefile, which is in the local CRS, and convert it to EPSG 4326 (WGS 84) for compatibility with other data sources
study_area = f"./source/{place}_studyArea.shp"


In [3]:
# Parameters
local_crs = 26917
place = "atlanta"
latlng = "(33.748783, -84.388168)"
dist = 30000
study_area = "./source/atlanta_studyArea.shp"


In [4]:
# Read in the study area polygon shapefile, which is in the local CRS, and convert it to EPSG 4326 (WGS 84) for compatibility with other data sources
study_area = gpd.read_file(study_area).to_crs(epsg=4326)


# Calculate the union of all study area polygons to create a single polygon that covers the entire study area
study_area_polygon = study_area.geometry.unary_union


In [5]:
# Use the study area polygon to extract water geometries (i.e. bodies of water) from OpenStreetMap data using the `geometries_from_polygon` function from the `osmnx` library
# `tags` parameter specifies which OSM tags to include in the extraction (in this case, only include natural=water tags)
water = ox.geometries.geometries_from_polygon(study_area_polygon, tags={"natural": "water"}).reset_index(drop=True)

# Preview the first few rows of the resulting GeoDataFrame
water.head()


,geometry,ele,gnis:county_id,gnis:created,gnis:feature_id,gnis:state_id,name,natural,water,waterway,...,ship,fixme,addr:housenumber,addr:state,addr:street,source:name,boundary,attraction,operator,is_in
0,"POLYGON ((-84.38361 33.44442, -84.38371 33.444...",235.0,NaN,NaN,NaN,NaN,NaN,water,river,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"POLYGON ((-84.38361 33.44442, -84.38371 33.444...",231.0,NaN,NaN,NaN,NaN,NaN,water,river,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"POLYGON ((-84.46936 33.45498, -84.46950 33.455...",NaN,NaN,NaN,337950,NaN,Pye Lake,water,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"POLYGON ((-84.27556 33.21548, -84.27549 33.215...",267.00000000000,NaN,NaN,NaN,NaN,NaN,water,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"POLYGON ((-84.39327 33.36455, -84.39328 33.364...",230.0,NaN,NaN,NaN,NaN,NaN,water,river,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# Assuming 'gdf' is your GeoDataFrame
geometry_types = water.geometry.geom_type

# Create a mask for polygons and multipolygons
mask = (geometry_types == 'Polygon') | (geometry_types == 'MultiPolygon')

# Filter the GeoDataFrame
water = water[mask]

In [7]:
# Assuming 'gdf' is your GeoDataFrame
geometry_types = water.geometry.geom_type

# Get unique types
unique_geometry_types = geometry_types.unique()

# Print types
print(unique_geometry_types)

['Polygon' 'MultiPolygon']


In [8]:
#drop all non-geometry columns
def drop_columns(df):
    df = df.drop(df.columns.difference(['geometry']), 1)
    return df

In [9]:
# of a geodataframe that only have polyogns, extract all outlines into a geodataframe of linestrings

outlines = water.copy()
outlines["geometry"] = outlines.boundary.to_crs(local_crs)
outlines = outlines.drop(outlines.columns.difference(['geometry']), 1)

outlines.head()

outlines.to_parquet(f"./out/{place}/water_outlines.pq")

# outlines = outlines.loc[outlines.geometry.type == "LineString"]
# outlines = outlines.reset_index(drop=True)

/tmp/ipykernel_24600/2010371134.py:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  outlines = outlines.drop(outlines.columns.difference(['geometry']), 1)


In [13]:
# If study_area and water are lists of geometries
study_area_geometry = unary_union(study_area.geometry)
water_geometry = unary_union(water.geometry)

# Perform the difference operation
result = study_area_geometry.difference(water_geometry)

In [16]:
study_area_a = gpd.GeoDataFrame(geometry=[result])
study_area_a.crs = study_area.crs

study_area = study_area_a

In [ ]:
# # Overlay the `water` GeoDataFrame on top of the `study_area` polygon using the 'difference' method to remove the water geometries from the study area polygon
# study_area = study_area.overlay(water, how='difference')

In [ ]:
# Create a plot of the `study_area` polygon using the `plot()` method
study_area.plot()

In [ ]:
# Use the study area polygon to extract building geometries from OpenStreetMap data using the `geometries_from_polygon` function from the `osmnx` library
# `tags` parameter specifies which OSM tags to include in the extraction (in this case, only include building tags)
buildings = ox.geometries.geometries_from_polygon(study_area_polygon, tags={'building':True})

# Preview the first few rows of the resulting GeoDataFrame
buildings.head()

In [ ]:
# Select only building geometries that are valid polygons (i.e. exclude other geometry types like Points and MultiPolygons) and convert the GeoDataFrame to the local coordinate reference system
buildings = buildings[buildings.geom_type == "Polygon"].reset_index(drop=True)
buildings = buildings[["geometry"]].to_crs(local_crs)

# Print the count of each geometry type to check that only Polygon geometries remain
print(buildings.geom_type.value_counts())

In [ ]:
# Merge adjacent or overlapping building polygons using the `unary_union()` method
merged = buildings.geometry.unary_union

# Convert the merged geometry back to a GeoDataFrame with a single polygon
merged_buildings_gdf = gpd.GeoDataFrame(geometry=[merged])

# Explode the GeoDataFrame to convert the single polygon back into multiple separate polygons
buildings = merged_buildings_gdf.explode()

# Select only building geometries that are valid polygons (i.e. exclude other geometry types like Points and MultiPolygons)
buildings = buildings[buildings.geom_type == "Polygon"].reset_index(drop=True)


In [ ]:
# Add a new column to the GeoDataFrame called "uID" containing a range of values from 0 to the length of the GeoDataFrame (minus 1)
buildings["uID"] = range(len(buildings))

In [ ]:
# Print out the count of each type of geometry in the GeoDataFrame
print(buildings.geom_type.value_counts())

# Display the first few rows of the GeoDataFrame
buildings.head()

In [ ]:
osm_graph= ox.graph.graph_from_polygon(study_area_polygon, network_type='drive')
osm_graph = ox.projection.project_graph(osm_graph, to_crs=local_crs)
streets = ox.graph_to_gdfs(
    osm_graph,
    nodes=False,
    edges=True,
    node_geometry=False,
    fill_edge_geometry=True
)

streets.head()

In [ ]:
# create a new column in the streets GeoDataFrame called 'motorway' that is 1 if the 'highway' column contains 'motorway', 'trunk', 'motorway_link' or 'trunk_link' and 0 otherwise
def is_motorway(highway):
    if isinstance(highway, list):
        return 1 if any(x in ['motorway', 'trunk', 'motorway_link', 'trunk_link'] for x in highway) else 0
    else:
        return 1 if highway in ['motorway', 'trunk', 'motorway_link', 'trunk_link'] else 0
streets["is_motorway"] = streets["highway"].apply(is_motorway)

def is_primary(highway):
    if isinstance(highway, list):
        return 1 if any(x in ['primary', 'primary_link'] for x in highway) else 0
    else:
        return 1 if highway in ['primary', 'primary_link'] else 0
    
def is_link(highway):
    if isinstance(highway, list):
        return 1 if any(x in ['motorway_link', 'trunk_link', "primary_link", "secondary_link", "tertiary_link"] for x in highway) else 0
    else:
        return 1 if highway in ['motorway_link', 'trunk_link', "primary_link", "secondary_link", "tertiary_link"] else 0
    
def is_roundabout(junction, highway):
    
    if isinstance(junction, list):
        if any(x in ['roundabout', 'circular'] for x in junction):
            return 1
    else:
        if junction in ['roundabout', 'circular']:
            return 1
    
    if isinstance(highway, list):
        return 1 if any(x in ['mini_roundabout'] for x in highway) else 0
    else:
        return 1 if junction in ['roundabout'] else 0

In [ ]:
# get all the unique values of the 'highway' column of streets

highway_types = set()
for highway in streets["highway"]:
    if isinstance(highway, list):
        for h in highway:
            highway_types.add(h)
    else:
        print(highway)
        highway_types.add(highway)

# assign a key to each street type in highway_types
key = {types:key+1 for key, types in enumerate(highway_types)}
key["residential"] = 0
key["living_street"] = 0
key["unclassified"] = 0

In [ ]:
print(key)

In [ ]:
streets["highway_types"] = streets["highway"].apply(lambda x: key[x] if isinstance(x, str) else key[x[0]])

In [ ]:
streets["is_motorway"] = streets["highway"].apply(is_motorway)
streets["is_primary"] = streets["highway"].apply(is_primary)
streets["is_link"] = streets["highway"].apply(is_link)
streets["is_roundabout"] = streets.apply(lambda x: is_roundabout(x["junction"], x["highway"]), axis=1)
streets["all_ones"] = 1

In [ ]:
streets["road_char_field"] = streets.apply(lambda row: 0 if row["is_roundabout"] == 1 else (2 if row["is_link"] == 1 else 1), axis=1)

In [ ]:
streets.head()

In [ ]:
buildings.plot()

In [ ]:
study_area = study_area.to_crs(local_crs)
study_area.plot()

In [ ]:
## create directory ./out/{place} if it does not exist
def create_dir(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
        
create_dir("./out/{place}")

In [ ]:
## convert streets_noded_gdf, buildings, and study_area to local_crs

buildings.to_parquet(f"./out/{place}/buildings_raw.pq")

study_area.to_parquet(f"./out/{place}/study_area.pq")

# streets.to_parquet("./out/{place}/streets.pq")

In [ ]:
water.drop(columns=water.columns.difference(['geometry'])).to_crs(local_crs).reset_index(drop=True).to_parquet(f"./out/{place}/water.pq")

In [ ]:
# iterate through the columns of streets, if it is a list, cast it as a string

for col in streets.columns:
    for i, row in streets.iterrows():
        if isinstance(row[col], list):
            streets.loc[i, col] = ','.join(str(streets.loc[i, col]))

In [ ]:
import os
import glob

# define the directory path
directory_path = f"./out/{place}/"

# define the pattern to match the files you want to delete
pattern = directory_path + "streets_raw*"

# use glob to get a list of files that match the pattern
file_list = glob.glob(pattern)

# loop through the list of files and delete each file
for file_path in file_list:
    try:
        os.remove(file_path)
        print(f"{file_path} has been deleted.")
    except OSError:
        print(f"Error while deleting file: {file_path}")

In [ ]:
# save streets to shapefile
streets.to_file(f"./out/{place}/streets_raw.shp", driver='ESRI Shapefile')

In [ ]:
# # Create a polygon geodataframe from creating a 2 meter buffer around every line in streets_noded_gdf and dissolve it into study_area

# streets_noded_gdf_buffer = dgpd.from_geopandas(streets_noded_gdf, npartitions=4)
# streets_noded_gdf_buffer.buffer(2)

# study_area_dgpd = dgpd.from_geopandas(study_area, npartitions=4).append(streets_noded_gdf_buffer).dissolve()

In [ ]:
# test = study_area_dgpd.compute()

In [ ]:
# # dissolve streets_noded_gdf_buffer into study_area into one multipolygon in a geodataframe
# concat = pd.concat([study_area, streets_noded_gdf_buffer])

# study_area_polygon = gdf.geometry.unary_union
# study_area = gpd.GeoDataFrame(geometry=[dissolved_geom], crs=gdf.crs)

In [ ]:
# enclosures = momepy.enclosures(noded_gdf , limit= study_area.to_crs(local_crs))

In [ ]:
# enclosures.plot()

In [ ]:
# # Perform a spatial join of the overlapping polygons with themselves
# spatial_join = gpd.sjoin(enclosures, enclosures, how="inner", op="intersects")

# # Count the number of overlapping polygons for each polygon
# overlapping_counts = spatial_join.groupby(["eID_left"]).size()

# # Get the polygons that overlap with more than one other polygon
# overlapping_count = overlapping_counts[overlapping_counts > 30].index.tolist()
